In [1]:
#!pip install xgboost

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

In [3]:
dataset = pd.read_csv("asthma_disease_data.csv", index_col=None)
df2 = dataset
df2 = pd.get_dummies(df2, drop_first=True)

indep_X = df2.drop(['PatientID','Ethnicity','EducationLevel','PhysicalActivity','GastroesophagealReflux'], axis=1)
dep_Y = df2['Diagnosis']

In [4]:
dataset

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,PhysicalActivity,DietQuality,SleepQuality,...,LungFunctionFEV1,LungFunctionFVC,Wheezing,ShortnessOfBreath,ChestTightness,Coughing,NighttimeSymptoms,ExerciseInduced,Diagnosis,DoctorInCharge
0,5034,63,0,1,0,15.848744,0,0.894448,5.488696,8.701003,...,1.369051,4.941206,0,0,1,0,0,1,0,Dr_Confid
1,5035,26,1,2,2,22.757042,0,5.897329,6.341014,5.153966,...,2.197767,1.702393,1,0,0,1,1,1,0,Dr_Confid
2,5036,57,0,2,1,18.395396,0,6.739367,9.196237,6.840647,...,1.698011,5.022553,1,1,1,0,1,1,0,Dr_Confid
3,5037,40,1,2,1,38.515278,0,1.404503,5.826532,4.253036,...,3.032037,2.300159,1,0,1,1,1,0,0,Dr_Confid
4,5038,61,0,0,3,19.283802,0,4.604493,3.127048,9.625799,...,3.470589,3.067944,1,1,1,0,0,1,0,Dr_Confid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,7421,43,1,0,2,29.059613,0,3.019854,6.119637,8.300960,...,3.125249,5.166032,0,1,0,0,0,1,1,Dr_Confid
2388,7422,18,1,0,1,20.740850,0,5.805180,4.386992,7.731192,...,1.132977,5.509502,0,0,0,1,1,0,1,Dr_Confid
2389,7423,54,0,3,2,37.079560,0,4.735169,8.214064,7.483521,...,1.685962,3.346877,1,0,1,1,0,1,1,Dr_Confid
2390,7424,46,1,0,2,23.444712,0,9.672637,7.362861,6.717272,...,3.481549,1.713274,0,1,1,0,1,1,0,Dr_Confid


In [5]:
dataset.columns

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'PhysicalActivity', 'DietQuality', 'SleepQuality',
       'PollutionExposure', 'PollenExposure', 'DustExposure', 'PetAllergy',
       'FamilyHistoryAsthma', 'HistoryOfAllergies', 'Eczema', 'HayFever',
       'GastroesophagealReflux', 'LungFunctionFEV1', 'LungFunctionFVC',
       'Wheezing', 'ShortnessOfBreath', 'ChestTightness', 'Coughing',
       'NighttimeSymptoms', 'ExerciseInduced', 'Diagnosis', 'DoctorInCharge'],
      dtype='object')

In [6]:
columns_to_drop=['PatientID','Ethnicity','EducationLevel','PhysicalActivity','GastroesophagealReflux','PhysicalActivity',
                    'DietQuality','SleepQuality']
dataset=dataset.drop(columns=columns_to_drop)

In [7]:
indep_X = df2.drop(['PatientID','Ethnicity','EducationLevel','PhysicalActivity','GastroesophagealReflux','PhysicalActivity',
                    'DietQuality','SleepQuality'], axis=1)
dep_Y = df2['Diagnosis']
dataset

,Age,Gender,BMI,Smoking,PollutionExposure,PollenExposure,DustExposure,PetAllergy,FamilyHistoryAsthma,HistoryOfAllergies,...,LungFunctionFEV1,LungFunctionFVC,Wheezing,ShortnessOfBreath,ChestTightness,Coughing,NighttimeSymptoms,ExerciseInduced,Diagnosis,DoctorInCharge
0,63,0,15.848744,0,7.388481,2.855578,0.974339,1,1,0,...,1.369051,4.941206,0,0,1,0,0,1,0,Dr_Confid
1,26,1,22.757042,0,1.969838,7.457665,6.584631,0,0,1,...,2.197767,1.702393,1,0,0,1,1,1,0,Dr_Confid
2,57,0,18.395396,0,1.460593,1.448189,5.445799,0,1,1,...,1.698011,5.022553,1,1,1,0,1,1,0,Dr_Confid
3,40,1,38.515278,0,0.581905,7.571845,3.965316,0,0,0,...,3.032037,2.300159,1,0,1,1,1,0,0,Dr_Confid
4,61,0,19.283802,0,0.980875,3.049807,8.260605,0,0,0,...,3.470589,3.067944,1,1,1,0,0,1,0,Dr_Confid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,43,1,29.059613,0,2.483829,7.314582,3.425445,1,0,1,...,3.125249,5.166032,0,1,0,0,0,1,1,Dr_Confid
2388,18,1,20.740850,0,7.733983,2.279073,6.467701,0,0,0,...,1.132977,5.509502,0,0,0,1,1,0,1,Dr_Confid
2389,54,0,37.079560,0,2.794847,3.055139,9.484013,0,0,0,...,1.685962,3.346877,1,0,1,1,0,1,1,Dr_Confid
2390,46,1,23.444712,0,9.448862,7.712584,5.051405,0,1,1,...,3.481549,1.713274,0,1,1,0,1,1,0,Dr_Confid


In [8]:
def split_scalar(indep_X,dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)    
    return X_train, X_test, y_train, y_test

In [9]:
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [10]:
def Linear(X_train,y_train,X_test):       
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return  r2 

In [11]:
def Decision(X_train,y_train,X_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return  r2 

In [12]:
def random(X_train,y_train,X_test):       
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return  r2

In [13]:
def xgboost(X_train,y_train,X_test):       
    from xgboost import XGBRegressor
    regressor = XGBRegressor(n_jobs=5,learning_rate=0.1,max_depth=10,random_state=1)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test)
    return r2

In [14]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []  
    # List to store column names for each model
    r2_values = []  
    # List to store R2 values for each model

    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()

    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state=0)

    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators=10, random_state=0)

    from xgboost import XGBRegressor
    xgb = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)

        # Get the column names selected by RFE
        selected_columns = [col for col, selected in zip(indep_X.columns, log_rfe.support_) if selected]
        colnames_list.append(selected_columns)

        # Fit the model and calculate and store the R2 value
        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)
        model.fit(X_train, y_train)  # Fit the model
        r2 = r2_prediction(model, X_test, y_test)
        r2_values.append(r2)

    return rfelist, colnames_list, r2_values

# Call the function with your data
rfelist, colnames_list, r2_values = rfeFeature(indep_X, dep_Y, 5)

# Print the selected column names and R2 values for each model
for model_name, selected_columns, r2_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, r2_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"R2 Value: {r2_value}\n")

Model: Linear
Selected Columns: ['Eczema', 'LungFunctionFEV1', 'ShortnessOfBreath', 'Coughing', 'Diagnosis']
R2 Value: 1.0

Model: Decision
Selected Columns: ['ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced', 'Diagnosis']
R2 Value: 1.0

Model: Random
Selected Columns: ['ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced', 'Diagnosis']
R2 Value: 1.0

Model: XGBoost
Selected Columns: ['ChestTightness', 'Coughing', 'NighttimeSymptoms', 'ExerciseInduced', 'Diagnosis']
R2 Value: 0.999999986390663



# Model creation

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import pickle

selected_features=indep_X
target_variable=dep_Y
rfelist = selected_features  # Replace with your selected features
dep_Y = target_variable  # Replace with your target variable

# Now you can use rfelist and colnames_list to train the Decision Tree model with the selected features
# Perform GridSearchCV for hyperparameter tuning
X_train, X_test, y_train, y_test = train_test_split(rfelist, dep_Y, test_size=0.25, random_state=0)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
param_grid={'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
grid=GridSearchCV(SVR(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(X_train,y_train)
re=grid.cv_results_
y_predict=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predict)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test,y_predict)
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_predict,average='weighted')
print("The f1 macro value the best parameter{}:".format(grid.best_params_),f1_macro)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
print(cm)